# Altering Tables
One SQL feature that BigQuery doesn't support (for good reasons) is `ALTER TABLE`. But there many valid cases you might want to do that. The only way to do that now is:
1. Create a new table with a query
2. Delete the old table
3. Rename the new table to replace the old one

Problem here is this can take a long time if you want to apply it to all partitions of a table. So we wrote a tool to do just this in an automated way.

## Let's go over an example
As always, we first create a `BqT` object:

In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../../")
sys.path.insert(0, "../../src")

from bqt import bqt

bqt.change_project("zsc-personal")  # note how we can change a project

Now let's create a test table to alter later. We're intentionally creating them as two partitions to show how this would work for many table partitions.

In [2]:
bqt.verbose = True

In [3]:
bqt.create_table(
    """
SELECT 'user_1' AS user_id,
       'user_1@spotify.net' AS emal,
       '2017-01-01' AS registration_date,
       '2000-01-01' AS birthday
""",
    dataset="bqt_playground",
    table="my_profiles_20180201",
    write_disposition="WRITE_TRUNCATE",
)

bqt.create_table(
    """
SELECT 'user_2' AS user_id,
       'user_2@spotify.net' AS emal,
       '2018-01-01' AS registration_date,
       '2010-01-01' AS birthday
""",
    dataset="bqt_playground",
    table="my_profiles_20180202",
    write_disposition="WRITE_TRUNCATE",
)

Creating table `zsc-personal.bqt_playground.my_profiles_20180201` ...
Running job zsc-personal:7f0be397-7410-4bf9-bf4c-d1b31b1d7b44
Job finished successfully!                   
Creating table `zsc-personal.bqt_playground.my_profiles_20180202` ...
Running job zsc-personal:6e8312ff-f3a7-4f3f-a93c-45d9cd525144
Job finished successfully!                   


In [4]:
bqt.create_table?

Signature:
bqt.create_table(
    query,
    dataset,
    table,
    project=None,
    dialect='standard',
    write_disposition='WRITE_EMPTY',
    ignore_exists=False,
    skip_formatting=False,
    schema=None,
    description=None,
    confirm=True,
    jinja=True,
)
Docstring:
Create a table and blocks until it's finished

Args:
    query (string): query to run
    dataset (string): dataset of destimatopm table
    table (string): destination table name
    dialect (string): 'legacy' or 'standard', use standard ;)
    write_disposition (string): one of the supported BigQuery
        write disposition types
    ignore_exists (bool): ignore the table if it exists
    schema (list of dictionaries): in format from BqSingleTable().schema
        Example:
        [{'description': u'Genre GID in hexadecimal string',
          'mode': u'REQUIRED',
          'name': u'genre_gid',
          'type': u'STRING'},
         {'description': u'Date of entity creation',
          'mode': u'REQUIRED',

Alright, now let's say we want to make the following two changes:
1. Because of GDPR, we can't store birthday, so let's change that to age
2. All registration dates were stored under the wrong type (`STRING`) we should make them into `DATE`
3. `email` was spelled `emal`, should fix that

In [ ]:
# first tell bqt what table or tables we want to alter
alter_obj = bqt.alter_table(
    dataset="bqt_playground",
    table="my_profiles_YYYYMMDD",  # note that we're passing the table format to capture all partitions
)

# now to do (1)
alter_obj.add_column("age", "DATE_DIFF(CURRENT_DATE, CAST(birthday AS DATE), YEAR)")  # add age based on birthday
alter_obj.delete_column("birthday")  # delete birthday

# and (2)
alter_obj.cast_column("registration_date", "DATE")

# and (3)
alter_obj.rename_column("emal", "email")

So far, we just defined our changes, it's always good to double check them before actually applying them:

In [ ]:
print(alter_obj.human_summary())

Looks good, let's apply:

In [ ]:
alter_obj.apply()

And to be sure that it did what it said it will do:

In [ ]:
bqt.query("SELECT * FROM `data-science-golden-path.bqt_playground.my_profiles_*`")